In [10]:
import os
import numpy as np
import matplotlib as mpl
mpl.use('Agg')  # Use a non-interactive backend
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import DataLoader


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/opt/anaconda3/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/opt/anaconda3/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 701, in start
    self.io_loop.start()
  File "/opt/anaconda3/lib/python3.12/site-

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: initialization failed

In [11]:
def generate_age_variation(model, image, cond, age_values):
    """
    Given an image and its condition, encode it and then decode it
    with varying age conditions.

    Args:
        model: Trained ConditionalVAE.
        image: A single image tensor (C x H x W).
        cond: Its corresponding condition tensor (age), shape [1].
        age_values: Iterable of new normalized age values.

    Returns:
        List of generated images (tensors).
    """
    model.eval()
    outputs = []
    with torch.no_grad():
        image = image.unsqueeze(0).to(device)
        cond = cond.unsqueeze(0).to(device)

        mu, logvar = model.encoder(image, cond)
        z = reparameterize(mu, logvar)
        for age in age_values:
            new_cond = torch.tensor([[age]], dtype=torch.float32).to(device)
            out = model.decoder(z, new_cond)
            outputs.append(out)
    return outputs

# Load a specific image called "image1" without using torchvision
import os
from PIL import Image
import numpy as np

# Assuming image1 is in the same directory or specify the path
image_path = "image1.jpg"  # Adjust path as needed

# Check if file exists
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Could not find {image_path}. Please ensure the file exists.")

# Load and manually preprocess the image using PIL and numpy
img = Image.open(image_path).convert('RGB')
img = img.resize((64, 64), Image.Resampling.LANCZOS)
img_np = np.array(img) / 255.0  # Normalize to [0,1]

# Convert to PyTorch tensor and normalize to [-1,1]
img_tensor = torch.from_numpy(img_np.transpose(2, 0, 1)).float()  # HWC to CHW format
img_tensor = img_tensor * 2.0 - 1.0  # Convert [0,1] to [-1,1]

# For the condition (age), we'll need to provide a value
sample_cond = torch.tensor([0.5], dtype=torch.float32)  # Default to middle age (0.5)

# Generate age variations
age_range = np.linspace(0.0, 1.0, 10)
generated_images = generate_age_variation(model, img_tensor, sample_cond, age_range)

# Display the results
plt.figure(figsize=(15, 3))
# First show the original image
img_display = (img_tensor.cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
plt.subplot(1, len(generated_images)+1, 1)
plt.imshow(img_display)
plt.title(f"Original (Age: {sample_cond.item():.2f})")
plt.axis("off")

# Then show the age variations
for i, gen in enumerate(generated_images):
    gen_np = (gen.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
    plt.subplot(1, len(generated_images)+1, i+2)
    plt.imshow(gen_np)
    plt.title(f"Age: {age_range[i]:.2f}")
    plt.axis("off")
plt.show()

FileNotFoundError: Could not find image1.jpg. Please ensure the file exists.

In [ ]:
def generate_age_variation(model, image, cond, age_values):
    """
    Given an image and its condition, encode it and then decode it
    with varying age conditions.

    Args:
        model: Trained ConditionalVAE.
        image: A single image tensor (C x H x W).
        cond: Its corresponding condition tensor (age), shape [1].
        age_values: Iterable of new normalized age values.

    Returns:
        List of generated images (tensors).
    """
    model.eval()
    outputs = []
    with torch.no_grad():
        image = image.unsqueeze(0).to(device)
        cond = cond.unsqueeze(0).to(device)

        mu, logvar = model.encoder(image, cond)
        z = reparameterize(mu, logvar)
        for age in age_values:
            new_cond = torch.tensor([[age]], dtype=torch.float32).to(device)
            out = model.decoder(z, new_cond)
            outputs.append(out)
    return outputs

# Load a specific image called "image1" without using torchvision
import os
from PIL import Image
import numpy as np

# Assuming image1 is in the same directory or specify the path
image_path = "image1.jpg"  # Adjust path as needed

# Check if file exists
if not os.path.exists(image_path):
    raise FileNotFoundError(f"Could not find {image_path}. Please ensure the file exists.")

# Load and manually preprocess the image using PIL and numpy
img = Image.open(image_path).convert('RGB')
img = img.resize((64, 64), Image.Resampling.LANCZOS)
img_np = np.array(img) / 255.0  # Normalize to [0,1]

# Convert to PyTorch tensor and normalize to [-1,1]
img_tensor = torch.from_numpy(img_np.transpose(2, 0, 1)).float()  # HWC to CHW format
img_tensor = img_tensor * 2.0 - 1.0  # Convert [0,1] to [-1,1]

# For the condition (age), we'll need to provide a value
sample_cond = torch.tensor([0.5], dtype=torch.float32)  # Default to middle age (0.5)

# Generate age variations
age_range = np.linspace(0.0, 1.0, 10)
generated_images = generate_age_variation(model, img_tensor, sample_cond, age_range)

# Display the results
plt.figure(figsize=(15, 3))
# First show the original image
img_display = (img_tensor.cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
plt.subplot(1, len(generated_images)+1, 1)
plt.imshow(img_display)
plt.title(f"Original (Age: {sample_cond.item():.2f})")
plt.axis("off")

# Then show the age variations
for i, gen in enumerate(generated_images):
    gen_np = (gen.squeeze().cpu().numpy().transpose(1, 2, 0) * 0.5 + 0.5)
    plt.subplot(1, len(generated_images)+1, i+2)
    plt.imshow(gen_np)
    plt.title(f"Age: {age_range[i]:.2f}")
    plt.axis("off")
plt.show()